In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder,OneHotEncoder
import pickle

In [2]:
data = pd.read_csv("Churn_Modelling.csv")
data=data.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [3]:
label_encoder_gender = LabelEncoder()
data["Gender"] = label_encoder_gender.fit_transform(data["Gender"])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [4]:
Onehot_encoder_geo = OneHotEncoder()
geo_encoder = Onehot_encoder_geo.fit_transform(data[["Geography"]])
Onehot_encoder_geo.get_feature_names_out(["Geography"])
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns=Onehot_encoder_geo.get_feature_names_out(["Geography"]))
data = pd.concat([data.drop(["Geography"],axis=1),geo_encoded_df],axis=1)


In [5]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [6]:
#Divide Data Set 
X= data.drop(["EstimatedSalary"], axis=1)
y=data['EstimatedSalary']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=43)

scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

In [7]:
##Save Encoders and Scalers
with open("label_encoder_gender.pkl",'wb') as file:
    pickle.dump(label_encoder_gender,file)
with open("Onehot_encoder_geo.pkl",'wb') as file:
    pickle.dump(Onehot_encoder_geo,file)
with open("scalar.pkl",'wb') as file:
    pickle.dump(scalar,file)

# ANN with Regression Problem Statement

In [8]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

model = Sequential([ 
                        Dense(64, activation='relu',input_shape=(X_train.shape[1],)),
                        Dense(32, activation='relu'),
                        Dense(1)
                    ]
)

c:\Users\mrafi\Documents\ANN\ANN\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.MeanAbsoluteError()
model.compile(optimizer=optimizer, loss=loss, metrics=["mae"])

In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
##Set up TensorBoard
import datetime
log_dir = "regressionlogs/fit"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#tensorflow_callbacks = TensorBoard(log_dir=log_dir, histogram_freq=1)
import os 
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(log_dir, 'ckpt-{epoch}',".weights.h5"),
    save_weights_only=True,
    save_best_only=False,
    verbose=1
)
tensorflow_callbacks = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,  # Log histograms every epoch
    write_graph=True,  # Log the graph
    update_freq='epoch'  # How often to write logs (batch or epoch)
)
### Set up early stopping 
early_stopping_callbacks = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [19]:

###Train Model 
history = model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=100, callbacks=[tensorflow_callbacks,early_stopping_callbacks,checkpoint_callback])

Epoch 1/100
246/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 50083.2930 - mae: 50083.2930
Epoch 1: saving model to regressionlogs/fit20240826-182025\ckpt-1\.weights.h5
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 50079.9180 - mae: 50079.9180 - val_loss: 49340.6680 - val_mae: 49340.6680
Epoch 2/100
225/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 49801.4883 - mae: 49801.4883
Epoch 2: saving model to regressionlogs/fit20240826-182025\ckpt-2\.weights.h5
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 49802.3711 - mae: 49802.3711 - val_loss: 49205.9609 - val_mae: 49205.9609
Epoch 3/100
224/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 49835.9258 - mae: 49835.9258
Epoch 3: saving model to regressionlogs/fit20240826-182025\ckpt-3\.weights.h5
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 49832.4453 - mae: 49832.4453 - val_loss: 49169.7656 - val_mae: 49169.7656
Epoch 4/100
244/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 49454.8086 - mae: 49454.8086
Epoch 4: saving model to regressionlo

In [25]:
model.save("regressionmodel.h5")

In [16]:
# Load Tensorboard extension 
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [23]:
%tensorboard --logdir regressionlogs/fit20240826-182025

Reusing TensorBoard on port 6007 (pid 3704), started 0:00:03 ago. (Use '!kill 3704' to kill it.)

In [24]:
test_loss,test_mae = model.evaluate(X_test,y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 49111.6289 - mae: 49111.6289
